In [46]:
import os
from os import listdir
from os.path import isfile, join
import sys
import pandas as pd
import random
import numpy as np
import pickle as pkl
from operator import add 

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression, Lasso
from collections import Counter

from utils import *

import TCA

import copy

In [16]:
def prepare_data(project, metric):
    data_path = '../data/700/merged_data_original/' + project + '.csv'
    data_df = pd.read_csv(data_path)
    data_df.rename(columns = {'Unnamed: 0':'id'},inplace = True)

    for col in ['id', 'commit_hash', 'release']:
        if col in data_df.columns:
            data_df = data_df.drop([col], axis = 1)
    data_df = data_df.dropna()
    y = data_df.Bugs
    X = data_df.drop(['Bugs'],axis = 1)
    if metric == 'process':
        X = X[['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
    'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
    'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
    'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr']]
    elif metric == 'product':
        X = X.drop(['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
    'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
    'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
    'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr'],axis = 1)
    else:
        X = X
    df = X
    df['Bugs'] = y
    return df

In [59]:
df = prepare_data('ASNE','process')

In [60]:
y = df.Bugs
_count_major = Counter(y)
_count_major

Counter({1: 33, 0: 675})

In [62]:
_count_major.most_common(1)[0][0]

0

In [40]:
test_data.sort_index().index

Index(['ABAGAIL', 'ASNE', 'AlphabetIndex-Fast-Scroll-RecyclerView',
       'Android-RateThisApp', 'Bolts-Android', 'CalendarFX', 'CircularReveal',
       'Droppy', 'DynamicGrid', 'ESPlorer', 'EasyNavigation',
       'EasyRefreshLayout', 'ExpandableTextView', 'FastDFS_Client',
       'HomeMirror', 'HwTxtReader', 'Launcher', 'MarkdownView', 'PatternLock',
       'RESTMock', 'Recyclerview-Gallery', 'Reservoir', 'RxActivityResult',
       'RxCache', 'RxFingerprint', 'RxHttpUtils', 'SandVXposed',
       'Space-Navigation-View', 'StepView', 'TapTargetView', 'Telecine',
       'android-floating-action-button', 'android-flowlayout',
       'android-mvp-architecture', 'android-pdfview', 'android-rss',
       'android-sliding-layer-lib', 'android-transcoder', 'binnavi',
       'bottomsheet', 'clean-status-bar', 'dbeaver', 'dcevm',
       'dockerfile-maven', 'egads', 'emoji-java', 'emojicon',
       'environment-dashboard', 'facebook-java-business-sdk', 'gcm',
       'geofire-java', 'ip2region', 

In [18]:
metric = 'process'
meta_data = pd.read_pickle('results/TCA/process/tca.pkl')
source_projects = meta_data.source.unique()
pseudo_source_projects = meta_data.target.unique()

cluster_data_loc = 'results/median_data/level_2/fold_' + str(0)
test_data = pd.read_pickle(cluster_data_loc + '/test_data.pkl')
target_projects = test_data.index.values.tolist()

selected_src_projects = copy.deepcopy(pseudo_source_projects)
selected_src_projects = set(selected_src_projects)
selected_src_projects = list(selected_src_projects - set(target_projects))

data_vectors = {}
for src in pseudo_source_projects:
    print(src)
    src_data = prepare_data(src, metric)
    src_data_vector = src_data.median().values.tolist()[:-1]
    data_vectors[src] = src_data_vector

train_X = []
for i in range(meta_data.shape[0]):
    if i%1000 == 0:
        print(i)
    src = meta_data.iloc[i,0]
    trg = meta_data.iloc[i,1]
    f = meta_data.iloc[i,5]
    pci_20 = meta_data.iloc[i,6]
    src_data_vector = data_vectors[src]
    trg_data_vector = data_vectors[trg]
    train_X.append(trg_data_vector + src_data_vector)


train_y_f = meta_data.f.values.tolist()
train_y_p = meta_data.pci_20.values.tolist()

ShapeOfView
swipe-button
SmoothProgressBar
JustAuth
Android-ReactiveLocation
gradle-retrolambda
UpdateChecker
aho-corasick
ion-java
SoLoader
Once
easygcm
reflections
databus
mqtt-client
emoji-java
StepView
Space-Navigation-View
CalendarFX
knife
otter
AlphabetIndex-Fast-Scroll-RecyclerView
dbeaver
android-mvp-architecture
modernizer-maven-plugin
Launcher
RxFingerprint
react-native-background-job
tinybus
EasyNavigation
ExpandableTextView
Reservoir
native-navigation
joda-time-android
otto
HomeMirror
emojicon
bottomsheet
seata-samples
DynamicGrid
moditect
clean-status-bar
Recyclerview-Gallery
SandVXposed
EasyRefreshLayout
HwTxtReader
android-floating-action-button
RxCache
TapTargetView
ASNE
binnavi
progressbar
pandora
RESTMock
poster-generater
gcm
Telecine
Bolts-Android
FastDFS_Client
wycheproof
facebook-java-business-sdk
MarkdownView
dcevm
libsignal-service-java
ABAGAIL
geofire-java
CircularReveal
libsignal-protocol-java
ESPlorer
android-transcoder
pcollections
PatternLock
RxHttpUtils
RxA

tx-lcn
subclipse
couchdb-lucene
TweetLanes
s3proxy
metacat
J2ME-Loader
react-native-iap
leshan
cucumber-reporting
unfolding
pippo
SeLion
600SeriesAndroidUploader
addressbook-level3
jcommander
athenz
materialistic
spring-cloud-consul
beetl2.0
addressbook-level4
sqlite-jdbc
Travel-Mate
HotswapAgent
kafka-connect-jdbc
spring-security-saml
spring-restdocs
OpenIAB
nginx-clojure
react-native-mapbox-gl
governator
android-beacon-library
bdd-security
google-java-format
fixflow
java-gitlab-api
andlytics
jdeb
samza
apicurio-studio
light-task-scheduler
RxJava-Android-Samples
super-csv
java-sdk
cassandra-unit
copycat
google-api-java-client
epoxy
Elephant
webcam-capture
angel
litemall
junit-plugin
Lightweight-Stream-API
phonegap-plugins
azure-spring-boot
wasabi
mosby
twoway-view
gitiles
extentreports-java
arquillian-core
manifold
JavaHamcrest
browser
SimianArmy
spring-boot-thin-launcher
Mallet
jongo
dsl-json
seldon-server
webbit
git-commit-id-maven-plugin
openstack-java-sdk
luwak
sempre
java-faker
s

In [21]:
clf_f = LinearRegression()
clf_p = LinearRegression()

clf_f.fit(train_X,train_y_f)
clf_p.fit(train_X,train_y_p)

LinearRegression()

In [31]:
median_data = pd.read_pickle('results/attributes/projects_median.pkl')

In [32]:
test_X = []
test_map = []
for sp in pseudo_source_projects:
    src_data_vector = median_data.loc[sp].values.tolist()
    for tp in target_projects:
        trg_data_vector = median_data.loc[tp].values.tolist()
        test_X.append(trg_data_vector + src_data_vector)
        test_map.append([sp, tp])

predicted_f = clf_f.predict(test_X)
predicted_p = clf_p.predict(test_X)

In [33]:
for i in range(len(predicted_f)):
    test_map[i].append(round(predicted_f[i],2))
    test_map[i].append(round(predicted_p[i],2))

final_result = test_map
final_result_df = pd.DataFrame(final_result, columns=['src','trg', 'f', 'pci_20'])

In [34]:
final_result_df

,src,trg,f,pci_20
0,ShapeOfView,environment-dashboard,0.63,0.68
1,ShapeOfView,plantuml,0.60,0.77
2,ShapeOfView,egads,0.62,0.66
3,ShapeOfView,dockerfile-maven,0.60,0.61
4,ShapeOfView,android-pdfview,0.63,0.65
...,...,...,...,...
60875,elasticsearch-jdbc,StepView,0.69,0.64
60876,elasticsearch-jdbc,Space-Navigation-View,0.66,0.64
60877,elasticsearch-jdbc,mqtt-client,0.72,0.66
60878,elasticsearch-jdbc,emoji-java,0.61,0.62


In [4]:
src_projects = []
source_p = set()
for trg in prediction_df.trg.unique():
    sub_df = prediction_df[prediction_df['trg'] == trg]
    sub_df.reset_index(inplace = True, drop = True)
    print(trg,sub_df.iloc[sub_df.f.idxmax()].src, sub_df.f.max())
    src_projects.append([trg,sub_df.iloc[sub_df.f.idxmax()].src,sub_df.iloc[sub_df.pci_20.idxmax()].src])
    source_p.add(sub_df.iloc[sub_df.pci_20.idxmax()].src)

ABAGAIL goclipse 1.07
ASNE goclipse 0.89
AlphabetIndex-Fast-Scroll-RecyclerView goclipse 1.05
Android-ReactiveLocation RustDT 1.05
Bolts-Android RustDT 1.08
CalendarFX goclipse 1.08
CircularReveal RustDT 1.05
Droppy RustDT 1.06
DynamicGrid RustDT 1.02
ESPlorer aws-sdk-android 0.17
EasyNavigation goclipse 1.05
EasyRefreshLayout fest-assert-2.x 0.97
ExpandableTextView RustDT 1.06
FastDFS_Client goclipse 1.06
HomeMirror goclipse 1.05
HwTxtReader goclipse 0.95
JustAuth goclipse 1.08
Launcher goclipse 1.06
MarkdownView goclipse 1.05
Once goclipse 1.06
PatternLock goclipse 1.05
RESTMock goclipse 1.05
Recyclerview-Gallery RustDT 1.05
Reservoir fest-assert-2.x 0.97
RxActivityResult goclipse 1.06
RxCache goclipse 1.06
RxFingerprint goclipse 1.07
RxHttpUtils goclipse 1.06
SandVXposed goclipse 1.04
ShapeOfView goclipse 1.05
SmoothProgressBar goclipse 1.06
SoLoader SoLoader 0.9
Space-Navigation-View RustDT 1.06
StepView RustDT 1.04
TapTargetView RustDT 1.06
Telecine goclipse 1.05
UpdateChecker goc

In [5]:
f1 = []
for project_pairs in src_projects:
    tgt_name, recall, pf, g, f, pci_20,precision, ifa = TCA.tca_plus_test(project_pairs[1:],[project_pairs[0]])
    print(tgt_name, recall, pf, g, f, pci_20, precision, ifa)
    f1.append(f)

ABAGAIL 0.02 0.0 0.02 0.04 0.8


KeyboardInterrupt: 

In [ ]:
f1

In [ ]:
def _replaceitem(x):
    if x >= 0.5:
        return 0.5
    else:
        return 0.0

myList = [0,0.8,0.77]
a = [x / 2 for x in myList]
b = list(map(_replaceitem, myList))

In [ ]:
c = list(map(add, a, b))

In [ ]:
c

In [ ]:
def _replaceitem2(x):
    if x >= 0.5:
        return 1
    else:
        return 0

In [ ]:
z = list(map(_replaceitem2, c))
z

In [ ]:
data = pd.read_csv('data.csv')
data = data.drop('Unnamed: 0', axis = 1)

In [ ]:
train_y = data.f
train_X = data.drop('f', axis = 1)

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR

In [ ]:
clf = SVR()
clf.fit(train_X,train_y)

In [8]:
path = 'results/median_data/level_2/fold_'

goals = ['recall', 'pf', 'f', 'pci_20', 'precision', 'ifa']
for goal in goals:
    with open('results/mixed_data/level_2/Stats/' + goal + '.txt', 'w') as f:
        for fold in range(10):
            data = pd.read_csv(path + str(fold) + '/2PTL_results.csv')
            data = data[goal]
            for key in data:
                f.write("%s " % key)